# Создание датафрейма

In [ ]:
from google.colab import drive # Подключаем google disk
drive.mount('/content/drive')
import numpy as np
import pandas as pd
import os
import cv2
import PIL
import zipfile
from skimage.metrics import structural_similarity as compare_ssim

Mounted at /content/drive


In [ ]:
!unzip "/content/drive/MyDrive/Stajirovka/dataset.zip" -d "/content/temp" # Рапаковываем наш датасет

Выходные данные были обрезаны до нескольких последних строк (5000).
 extracting: /content/temp/train/labels/1307.txt  
 extracting: /content/temp/train/labels/1308.txt  
 extracting: /content/temp/train/labels/1309.txt  
 extracting: /content/temp/train/labels/1310.txt  
 extracting: /content/temp/train/labels/1311.txt  
 extracting: /content/temp/train/labels/1312.txt  
 extracting: /content/temp/train/labels/1313.txt  
 extracting: /content/temp/train/labels/1314.txt  
 extracting: /content/temp/train/labels/1315.txt  
  inflating: /content/temp/train/labels/1316.txt  
  inflating: /content/temp/train/labels/1317.txt  
  inflating: /content/temp/train/labels/1318.txt  
  inflating: /content/temp/train/labels/1319.txt  
  inflating: /content/temp/train/labels/1320.txt  
  inflating: /content/temp/train/labels/1321.txt  
  inflating: /content/temp/train/labels/1322.txt  
  inflating: /content/temp/train/labels/1323.txt  
  inflating: /content/temp/train/labels/1324.txt  
  inflating: /

In [ ]:
!pip install ultralytics
from ultralytics import YOLO

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.7/303.7 KB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 17.5 MB/s eta 0:00:00


In [ ]:
# Берем модель Yolo и тренируем ее на coco
# Load a model
model = YOLO("yolov8n.yaml")  # build a new model from scratch
model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)

# Use the model
results = model.train(data="coco128.yaml", epochs=3)  # train the model
results = model.val()  # evaluate model performance on the validation set

In [ ]:
model = YOLO('/content/best.pt') # Подгружаем наши, ранее сохраненные веса

In [ ]:
result = model('/content/temp/val/images/2661.jpg') # Придикт


image 1/1 /content/temp/val/images/2661.jpg: 640x640 3 persons, 244.8ms
Speed: 2.1ms preprocess, 244.8ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 640)


In [ ]:
# Функция, которая отображает предсказания YOLO-модели
# На вход нужно подать путь к изображению, и аутпут нейронной сети
# П.С. - если вы работаете не в колабе, можно вывести результат, указав аргумент show=True в model.predict()
colors_list = ["red", "green", "blue"]
def show_predict(image_path, result):
  temp = []
  result = result[0].boxes.data.cpu()
  for j in result:
    if int(j[-1].detach().item()) == 0:
      temp.append(j)
  image = PIL.Image.open(image_path)
  draw_image = PIL.ImageDraw.Draw(image)
  for i in range(len(temp)):
    class_index = int(temp[i][-1].detach().item())
        # print(class_index)
    draw_image.rectangle(temp[i][:4].detach().numpy(), outline=colors_list[class_index], width=5)
  return image

In [ ]:
show_predict('/content/temp/val/images/2661.jpg', result) # Отображение фото с ббоксами

## Функции

In [ ]:
def getCrop(url, bbox):
  import cv2
  image = cv2.imread(url)
  x, y, w, h, conf, cls =  map(int, bbox)
  crop = image[y:h, x:w]
  filename = url.split("/")[-1]
  # img = cv2.imwrite(filename, crop)
  return crop, filename # На выходе получаем кропнутое фото в виде numpy масива, и имя файла

In [ ]:
def getCenterAndPosition(bbox):
    import math
    x1, y1, x2, y2, conf, cls = bbox.tolist()
    # находим центр баудинг бокса
    center_x = (x1 + x2) / 2
    center_y = (y1 + y2) / 2
    # Находим вектора позиции центра  - при условии что 640 на 640 это размер фото
    # Вектор A  - расстояние между началом координат 0 - 0 до центра ббокса
    # Вектор B  - расстояние между началом координат 0 - 640 до центра ббокса
    # Вектор C  - расстояние между началом координат 640 - 640 до центра ббокса
    # Вектор D  - расстояние между началом координат 640 - 0 до центра ббокса

    ax = 0
    ay = 0
    a = math.sqrt((center_x - ax)**2 + (center_y - ay)**2)

    bx = 0
    by = 0
    b = math.sqrt((center_x - bx)**2 + (center_y - by)**2)

    cx = 0
    cy = 0
    c = math.sqrt((center_x - cx)**2 + (center_y - cy)**2)

    dx = 0
    dy = 0
    d = math.sqrt((center_x - dx)**2 + (center_y - dy)**2)

    return center_x, center_y, a, b, c, d

In [ ]:
def getSSIM(image, image2):
    # Структурный индекс сходства (SSIM) - это метод,
    # который пытается измерить схожесть между двумя изображениями
    # с учетом их структуры и восприятия человеком.
    img1_resized = cv2.resize(image, (image2.shape[1], image2.shape[0]))

    gray_image = cv2.cvtColor(img1_resized, cv2.COLOR_BGR2GRAY)
    gray_image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
    ssim = compare_ssim(gray_image, gray_image2)
    return ssim

## Сборка датафрейма по только без ssim, psnr, mse, histDitance

In [ ]:
df = pd.DataFrame(columns=['file_name','center_x', 'center_y', 'a', 'b', 'c', 'd','x', 'y', 'w', 'h', 'ssim', 'psnr', 'mse', 'histDitance']) # Пустой датафрейм

In [ ]:
# Список названия файлов фотографий из датафрейма
# Взял из валидации но мжно из train
filenames = os.listdir('/content/temp/val/images') 
filenames.sort() # Сортировка по названием файлов

In [ ]:
# Собираем датафрейм, нашу таблицу
# Если в одном фрейме только один ббокс
# то работает условие else, try и.т.д
# если нету вообще ббокса, то выводим сообщение Нету ббокса
for i in filenames[:50]:
  result = model('/content/temp/val/images/' + i) # Придикт
  tensor = result[0].boxes.data # Получение тензора из придикта
  if tensor.shape[0] > 1: # Создаем условие если в одном фрейме больше одного ббокса
    for g in range(tensor.shape[0]):
      if int(tensor[g][-1].detach().item()) == 0: # Фильтруем только людей
        crop, filename = getCrop('/content/temp/val/images/' + i, tensor[g]) # Получаем кропнутую фото и имя файла
        filename_ = i.split('.')[0] + f'({g}).jpg' # Если в одном фрейме больше одного ббокса создаем имя файла с дополнительным скобками () полсе его названия
        center_x, center_y, a, b, c, d = getCenterAndPosition(tensor[g])
        x, y, w, h, conf, cls =  map(int, tensor[g])
        new_row = {'file_name':filename_,'center_x':center_x,'center_y':center_y,'a':a,'b':b,'c':c,'d':d,'x':x,'y':y,'w':w,'h':h}
        df = df.append(new_row, ignore_index=True) # Записываем в датафрейм
        img = cv2.imwrite('/content/temp2/' + filename_, crop) # Складываем кропнутые фото в папку temp2
      else:
        print('Не правильный класс')
  else:
    try:
      if int(tensor[0][-1].detach().item()) == 0:
        crop, filename = getCrop('/content/temp/val/images/' + i, tensor[0])
        center_x, center_y, a, b, c, d = getCenterAndPosition(tensor[0])
        x, y, w, h, conf, cls =  map(int, tensor[0])
        new_row = {'file_name':filename,'center_x':center_x,'center_y':center_y,'a':a,'b':b,'c':c,'d':d,'x':x,'y':y,'w':w,'h':h}
        df = df.append(new_row, ignore_index=True)
        img = cv2.imwrite('/content/temp2/' + filename, crop)
      else:
        print('Не правильный класс')
    except IndexError:
      print('Нету ббокса')

Ultralytics YOLOv8.0.49 🚀 Python-3.8.10 torch-1.13.1+cu116 CPU
Model summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs

image 1/1 /content/temp/val/images/2081.jpg: 640x640 3 persons, 329.2ms
Speed: 3.7ms preprocess, 329.2ms inference, 25.9ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/temp/val/images/2082.jpg: 640x640 1 person, 224.2ms
Speed: 3.0ms preprocess, 224.2ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/temp/val/images/2083.jpg: 640x640 1 person, 223.6ms
Speed: 1.2ms preprocess, 223.6ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/temp/val/images/2084.jpg: 640x640 1 person, 230.7ms
Speed: 3.3ms preprocess, 230.7ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/temp/val/images/2085.jpg: 640x640 1 person, 217.1ms
Speed: 3.4ms preprocess, 217.1ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)

image

Не правильный класс


image 1/1 /content/temp/val/images/2093.jpg: 640x640 1 person, 234.4ms
Speed: 3.1ms preprocess, 234.4ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/temp/val/images/2095.jpg: 640x640 (no detections), 340.1ms
Speed: 3.8ms preprocess, 340.1ms inference, 2.5ms postprocess per image at shape (1, 3, 640, 640)



Нету ббокса


image 1/1 /content/temp/val/images/2400.jpg: 640x640 1 person, 351.7ms
Speed: 1.4ms preprocess, 351.7ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/temp/val/images/2401.jpg: 640x640 1 person, 346.3ms
Speed: 3.7ms preprocess, 346.3ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/temp/val/images/2402.jpg: 640x640 1 person, 337.5ms
Speed: 1.3ms preprocess, 337.5ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/temp/val/images/2403.jpg: 640x640 1 person, 335.4ms
Speed: 1.4ms preprocess, 335.4ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/temp/val/images/2404.jpg: 640x640 (no detections), 334.6ms
Speed: 3.6ms preprocess, 334.6ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)



...

image 1/1 /content/temp/val/images/2463.jpg: 640x640 1 person, 198.9ms
Speed: 1.2ms preprocess, 198.9ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/temp/val/images/2464.jpg: 640x640 1 person, 197.4ms
Speed: 1.3ms preprocess, 197.4ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/temp/val/images/2465.jpg: 640x640 1 person, 1 refrigerator, 201.4ms
Speed: 1.2ms preprocess, 201.4ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)



Не правильный класс


image 1/1 /content/temp/val/images/2466.jpg: 640x640 1 person, 205.5ms
Speed: 1.2ms preprocess, 205.5ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/temp/val/images/2467.jpg: 640x640 1 person, 204.3ms
Speed: 1.2ms preprocess, 204.3ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/temp/val/images/2468.jpg: 640x640 1 person, 216.2ms
Speed: 1.3ms preprocess, 216.2ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/temp/val/images/2469.jpg: 640x640 1 person, 215.6ms
Speed: 1.3ms preprocess, 215.6ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/temp/val/images/2470.jpg: 640x640 1 person, 214.3ms
Speed: 1.2ms preprocess, 214.3ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/temp/val/images/2471.jpg: 640x640 1 person, 205.0ms
Speed: 3.0ms preprocess, 205.0ms inference, 1.4ms postprocess per image at shap

Нету ббокса


image 1/1 /content/temp/val/images/2478.jpg: 640x640 1 person, 218.2ms
Speed: 2.9ms preprocess, 218.2ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/temp/val/images/2479.jpg: 640x640 2 persons, 199.9ms
Speed: 1.2ms preprocess, 199.9ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/temp/val/images/2480.jpg: 640x640 1 person, 204.2ms
Speed: 1.4ms preprocess, 204.2ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/temp/val/images/2481.jpg: 640x640 1 person, 203.4ms
Speed: 1.2ms preprocess, 203.4ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/temp/val/images/2482.jpg: 640x640 2 persons, 197.8ms
Speed: 1.2ms preprocess, 197.8ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/temp/val/images/2483.jpg: 640x640 2 persons, 196.4ms
Speed: 1.9ms preprocess, 196.4ms inference, 1.4ms postprocess per image at s

In [ ]:
df

,file_name,center_x,center_y,a,b,...,h,ssim,psnr,mse,histDitance
0,2081(0).jpg,409.0,605.0,730.278029,730.278029,...,848,NaN,NaN,NaN,NaN
1,2081(1).jpg,366.5,896.0,968.059012,968.059012,...,1005,NaN,NaN,NaN,NaN
2,2081(2).jpg,275.0,896.5,937.729838,937.729838,...,1007,NaN,NaN,NaN,NaN
3,2082.jpg,394.5,637.0,749.265807,749.265807,...,884,NaN,NaN,NaN,NaN
4,2083.jpg,405.5,619.5,740.412385,740.412385,...,874,NaN,NaN,NaN,NaN
5,2084.jpg,402.0,622.5,741.019737,741.019737,...,874,NaN,NaN,NaN,NaN
6,2085.jpg,399.0,624.0,740.659841,740.659841,...,875,NaN,NaN,NaN,NaN
7,2086.jpg,398.0,624.0,740.121612,740.121612,...,874,NaN,NaN,NaN,NaN
8,2087.jpg,404.0,622.0,741.687266,741.687266,...,875,NaN,NaN,NaN,NaN
9,2088.jpg,420.5,635.5,762.023950,762.023950,...,906,NaN,NaN,NaN,NaN


In [ ]:
# Запаковка в архив
def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file), 
                       os.path.relpath(os.path.join(root, file), 
                                       os.path.join(path, '..')))

with zipfile.ZipFile('crop.zip', 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipdir('/content/temp2', zipf)